In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}


#extaraction le nombre des pages pour preparer l'extraction global 
url = 'https://www.avito.ma/fr/maroc/ventes_immobilieres-à_vendre'

page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text ,'html')
#extraction tout les balises <a> 
balises_a = soup.find_all('a',href=True)

#extraction le dernier nombre de pagination
nombres_entiers = []

for balise_a in balises_a:
    texte = balise_a.text.strip()
    if re.match(r'^-?\d+$', texte):
        nombres_entiers.append(int(texte))

lastpage = int(nombres_entiers[-1])
print(lastpage)

#extraction tout les lignes pour tout les annonces des voitures jusqu'à la deriniere page
all_liens =[]
for i in range(10):
    url = 'https://www.avito.ma/fr/maroc/ventes_immobilieres-à_vendre?o='+str(1)
    balises_a = soup.find_all('a',href=True,class_='sc-1jge648-0 eTbzNs')
    all_liens.extend(balises_a)
#tout les href qui rederige vers l'annonce 
hrefs = []
for link in all_liens:
    href = link.get('href')
    hrefs.append(href)

#tout les voitures ont extracté
liste_immobiliers = []
#pour chaque annonces seul
for href in hrefs :
    url = href
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text ,'html')
#les 3 premiers infos
    immoiler = {}
    first_infos = soup.find_all('span', attrs={'class': 'sc-1x0vz2r-0 kQHNss'})
    prixc = soup.find('p', attrs={'class': 'sc-1x0vz2r-0 lnEFFR sc-1g3sn3w-13 czygWQ'})
    if(prixc is None) :
        prixc = "NAN"
    else :
        prix = prixc.text
        prix_val= prix.replace(' ', '').replace(' ', '').replace('DH', '')
    values = []
    for info in first_infos :
        if (info is not None) :
            nombre = re.search(r'\d+', info.text).group()
            values.append(int(nombre))
        values.sort(reverse = True)
        if len(values) == 3 :
            metrage_nom = values[0]
            chambre_nbr = values[1]
            salle_de_bain_nbr = values[2]
        elif len(values) == 2 :
            if values[0] >10 :
                metrage_nom = values[0]
                chambre_nbr = values[1]
                salle_de_bain_nbr = "None"
            else :
                chambre_nbr = values[0]
                salle_de_bain_nbr = values[1]
        elif len(values) == 1 :
                if values[0] >10 :
                    metrage_nom = values[0]
                    salle_de_bain_nbr = "None"
                    chambre_nbr = "None"
                else :
                    metrage_nom = "None"
                    salle_de_bain_nbr = "None"
                    chambre_nbr = values[0]
        else :
                metrage_nom = "None"
                salle_de_bain_nbr = "None"
                chambre_nbr = "None"
    immoiler ["metrage_nom"] = metrage_nom
    immoiler ["salle_de_bain_nbr"] = salle_de_bain_nbr
    immoiler ["chambre_nbr"] = chambre_nbr
    immoiler ["prix en (DHS)"] = prix_val
#tout les noms "caracteristiques" et leur valeurs equivalents
    span_carac = soup.find_all('span',attrs={'class': 'sc-1x0vz2r-0 jZyObG'})
    span_carac_val = soup.find_all('span',attrs={'class': 'sc-1x0vz2r-0 gSLYtF'})
#compt pour chaque carac et sa valeur precis
    i = 0 
    for sc in span_carac :
        if sc.text == "Type":
            type = span_carac_val[i].text
            immoiler [sc.text] = type
        if sc.text == "Secteur":
            secteur = span_carac_val[i].text
            immoiler [sc.text] = secteur
        if sc.text == "Frais de syndic / mois":
            Frais_de_syndic = span_carac_val[i].text
            immoiler [sc.text] = Frais_de_syndic
        if sc.text == "Salons":
            salons = span_carac_val[i].text
            immoiler [sc.text] = salons 
        if sc.text == "Étage":
            etage = span_carac_val[i].text
            immoiler [sc.text] = etage
        if sc.text == "Surface habitable":
            surface = span_carac_val[i].text
        if sc.text == "Âge du bien":
            age_de_bien = span_carac_val[i].text
            immoiler [sc.text] = age_de_bien
        i += 1
    liste_immobiliers.append(immoiler)

#stocker les donnees extracte dans dataframe 
df = pd.DataFrame(liste_immobiliers)
display(df)
#save as csv
df.to_csv('avito_data_immobliers_structe121.csv', index=False , sep=';')  


1857


,metrage_nom,salle_de_bain_nbr,chambre_nbr,prix en (DHS),Type,Secteur,Âge du bien,Salons,Étage,Frais de syndic / mois
0,714,3,4,11000000,"Maisons et Villas, à vendre",Sidi Maarouf,11-20 ans,4,NaN,NaN
1,None,1,3,11000000,"Appartements, à vendre",Marjane,NaN,1,3,NaN
2,None,2,4,5600000,"Maisons et Villas, à vendre",Illigh,NaN,2,NaN,NaN
3,81,1,2,550000,"Appartements, à vendre",Kouilma,Neuf,2,1,49
4,54,1,1,900000,"Appartements, à vendre",Maarif,NaN,NaN,4,NaN
...,...,...,...,...,...,...,...,...,...,...
335,None,1,2,560000,"Appartements, à vendre",Toute la ville,Neuf,1,1,NaN
336,86,2,2,1400000,"Appartements, à vendre",Plage Oued Cherrat,6-10 ans,1,1,450
337,90,4,4,1670000,"Maisons et Villas, à vendre",Autre secteur,Neuf,4,NaN,NaN
338,62,1,2,380000,"Appartements, à vendre",Sidi Bernoussi,NaN,1,1,30
